In [1]:
!pip install pika

In [2]:
import pika
import json

# Параметры подключения
RABBITMQ_HOST = '192.168.0.18'  # или IP-адрес сервера
RABBITMQ_PORT = 8006  # ваш нестандартный порт
EXCHANGE_NAME = 'amq.topic'  # или имя вашего exchange
ROUTING_KEY = 'incoming_json'  # ключ маршрутизации
QUEUE_NAME = 'your_queue_name'  # имя очереди (если нужно)

# Тестовые данные
test_message = {
    "id": 123,
    "name": "Тестовое сообщение",
    "timestamp": "2024-01-15T10:30:00Z",
    "data": {
        "value": 42.5,
        "status": "success"
    }
}

def send_json_to_rabbitmq():
    try:
        # 1. Подключение к RabbitMQ
        credentials = pika.PlainCredentials('guest', 'guest')  # замените на свои
        parameters = pika.ConnectionParameters(
            host=RABBITMQ_HOST,
            port=RABBITMQ_PORT,
            credentials=credentials,
            heartbeat=600,
            blocked_connection_timeout=300
        )
        
        connection = pika.BlockingConnection(parameters)
        channel = connection.channel()
        
        # 2. Объявление exchange (если нужно)
        # channel.exchange_declare(
        #     exchange=EXCHANGE_NAME,
        #     exchange_type='topic',
        #     durable=True
        # )
        
        # 3. Отправка сообщения
        channel.basic_publish(
            exchange=EXCHANGE_NAME,  # используйте '' для default exchange
            routing_key=ROUTING_KEY,
            body=json.dumps(test_message, ensure_ascii=False),
            properties=pika.BasicProperties(
                content_type='application/json',
                delivery_mode=2,  # persistent message
                headers={'type': 'test_message'}
            )
        )
        
        print(f"✅ Сообщение отправлено в RabbitMQ ({RABBITMQ_HOST}:{RABBITMQ_PORT})")
        print(f"   Exchange: {EXCHANGE_NAME}")
        print(f"   Routing key: {ROUTING_KEY}")
        print(f"   Тело сообщения: {json.dumps(test_message, indent=2, ensure_ascii=False)}")
        
        # 4. Закрытие соединения
        connection.close()
        
    except Exception as e:
        print(f"❌ Ошибка при отправке сообщения: {str(e)}")



In [3]:
def check_rabbitmq_status():
    """Проверяет состояние RabbitMQ и наличие очередей"""
    try:
        credentials = pika.PlainCredentials('guest', 'guest')
        parameters = pika.ConnectionParameters(
            host=RABBITMQ_HOST,
            port=RABBITMQ_PORT,
            credentials=credentials,
            connection_attempts=3,
            retry_delay=2
        )
        
        connection = pika.BlockingConnection(parameters)
        channel = connection.channel()
        
        print("=== Статус RabbitMQ ===")
        print(f"Подключено к {RABBITMQ_HOST}:{RABBITMQ_PORT}")
        
        # Пытаемся получить информацию о сервере
        try:
            # Проверяем connection
            if connection.is_open:
                print("✅ Соединение установлено")
                
                # Создаем тестовую очередь, если не существует
                channel.queue_declare(queue='test_queue', durable=True, exclusive=False)
                
                # Привязываем к exchange amq.topic
                channel.queue_bind(
                    exchange='amq.topic',
                    queue='test_queue',
                    routing_key='incoming_json'
                )
                
                print("✅ Очередь 'test_queue' создана и привязана к amq.topic")
                print("✅ Routing key: 'incoming_json'")
                
                # Отправляем тестовое сообщение
                channel.basic_publish(
                    exchange='amq.topic',
                    routing_key='incoming_json',
                    body='{"test": "message"}',
                    properties=pika.BasicProperties(
                        delivery_mode=2,
                        content_type='application/json'
                    )
                )
                
                print("✅ Тестовое сообщение отправлено")
                
                # Получаем сообщение (чтобы очистить очередь)
                method_frame, header_frame, body = channel.basic_get(queue='test_queue')
                if method_frame:
                    print(f"✅ Получено сообщение: {body.decode()}")
                    channel.basic_ack(method_frame.delivery_tag)
                else:
                    print("❌ Сообщение не получено (возможно, не дошло до очереди)")
                    
        except Exception as e:
            print(f"❌ Ошибка при проверке: {e}")
            
        connection.close()
        
    except pika.exceptions.AMQPConnectionError as e:
        print(f"❌ Не удалось подключиться к RabbitMQ: {e}")
        print("Проверьте:")
        print(f"1. Доступен ли сервер {RABBITMQ_HOST}:{RABBITMQ_PORT}")
        print("2. Правильные ли учетные данные")
        print("3. Разрешен ли доступ с вашего IP")
    except Exception as e:
        print(f"❌ Общая ошибка: {e}")


check_rabbitmq_status()

=== Статус RabbitMQ ===
Подключено к 192.168.0.18:8006
✅ Соединение установлено
✅ Очередь 'test_queue' создана и привязана к amq.topic
✅ Routing key: 'incoming_json'
✅ Тестовое сообщение отправлено
✅ Получено сообщение: {"test": "message"}


In [4]:
send_json_to_rabbitmq()

✅ Сообщение отправлено в RabbitMQ (192.168.0.18:8006)
   Exchange: amq.topic
   Routing key: incoming_json
   Тело сообщения: {
  "id": 123,
  "name": "Тестовое сообщение",
  "timestamp": "2024-01-15T10:30:00Z",
  "data": {
    "value": 42.5,
    "status": "success"
  }
}


In [5]:
import socket
def diagnose_connection():
    host = "192.168.0.18"
    port = 8006
    
    print(f"Диагностика подключения к {host}:{port}")
    print("-" * 40)
    
    # 1. Проверка порта
    print("1. Проверка TCP порта...")
    try:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(5)
        result = sock.connect_ex((host, port))
        sock.close()
        
        if result == 0:
            print(f"   ✅ Порт {port} открыт")
        else:
            print(f"   ❌ Порт {port} закрыт (код: {result})")
            print(f"   Проверьте: docker ps | grep rabbitmq")
            return False
    except Exception as e:
        print(f"   ❌ Ошибка: {e}")
        return False
    
    # 2. Проверка RabbitMQ подключения
    print("\n2. Проверка RabbitMQ...")
    try:
        connection = pika.BlockingConnection(
            pika.ConnectionParameters(
                host=host,
                port=port,
                credentials=pika.PlainCredentials('guest', 'guest'),
                connection_attempts=2,
                retry_delay=1
            )
        )
        
        channel = connection.channel()
        print("   ✅ Подключение успешно")
        
        # Проверяем exchange
        try:
            channel.exchange_declare(
                exchange='amq.topic',
                exchange_type='topic',
                passive=True
            )
            print("   ✅ Exchange 'amq.topic' существует")
        except:
            print("   ⚠️ Exchange 'amq.topic' не найден")
        
        connection.close()
        return True
        
    except pika.exceptions.AMQPConnectionError as e:
        print(f"   ❌ Ошибка AMQP: {e}")
        return False
    except Exception as e:
        print(f"   ❌ Ошибка: {e}")
        return False




In [6]:
diagnose_connection()

Диагностика подключения к 192.168.0.18:8006
----------------------------------------
1. Проверка TCP порта...
   ✅ Порт 8006 открыт

2. Проверка RabbitMQ...
   ✅ Подключение успешно
   ✅ Exchange 'amq.topic' существует


True

In [11]:
import json
from datetime import datetime, timezone
import pika
from pika.exceptions import AMQPConnectionError, ChannelClosedByBroker


def push_maintenance_msg(type: str, owner: str, token: str, message: str):
    payload = {
        "o": owner,
        "t": token,
        "m": message,
        "dt": datetime.now(timezone.utc).isoformat(),
        "type": type,
    }

    try:
        connection = pika.BlockingConnection(
            pika.ConnectionParameters(
                host="192.168.0.18",
                port=8006,
                heartbeat=30,
                blocked_connection_timeout=10,
            )
        )
    except AMQPConnectionError as e:
        raise RuntimeError("Failed to connect to RabbitMQ") from e

    try:
        channel = connection.channel()

        channel.queue_declare(
            queue="incoming_json",
            durable=True
        )

        # включаем publisher confirms
        channel.confirm_delivery()

        published = channel.basic_publish(
            exchange="",
            routing_key="incoming_json",
            body=json.dumps(payload, ensure_ascii=False),
            properties=pika.BasicProperties(
                delivery_mode=2  # persistent
            ),
            mandatory=True
        )

        if not published:
            raise RuntimeError("Message was not confirmed by broker")

    except ChannelClosedByBroker as e:
        raise RuntimeError("Channel was closed by broker during publish") from e

    except Exception as e:
        raise RuntimeError(f"Failed to publish message e:{e}") from e

    finally:
        if connection and connection.is_open:
            connection.close()



In [12]:
test_type = "maintenance"
test_owner = "server_room_5"
test_token = "abc123-xyz789"
test_message = "Плановое техническое обслуживание будет проводиться 15.01.2024 с 23:00 до 03:00"

try:
    push_maintenance_msg(
        type=test_type,
        owner=test_owner,
        token=test_token,
        message=test_message
    )
    print("Сообщение успешно отправлено")
except RuntimeError as e:
    print(f"Ошибка при отправке: {e}")
except Exception as e:
    print(f"Неожиданная ошибка: {e}")

Ошибка при отправке: Failed to publish message e:Message was not confirmed by broker
